**Import PyVisa and check if installed**

In [1]:
import pyvisa
print(pyvisa.__version__)

1.14.1


In [ ]:
import math
def process_gcode(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue

            parts = line.split()
            command = parts[0]
            if parts[0] in ('G0', 'G1', 'G28', 'M204'):
                x = y = z = f = p = None

                for part in parts[1:]:
                    if part.startswith('X'):
                        x = float(part[1:])
                    elif part.startswith('Y'):
                        y = float(part[1:])
                    elif part.startswith('Z'):
                        z = float(part[1:])
                    elif part.startswith('F'):
                        f = float(part[1:])
                    elif part.startswith('P'):
                        p = float(part[1:])
                
                # Convert to polar coordinates if necessary
                x_polar, theta_polar = None, None
                if x is not None and y is not None:
                    x_polar = math.sqrt(x**2 + y**2)  # Radial distance
                    theta_polar = math.degrees(math.atan2(y, x))  # Angle in degrees
                
                handle_command(command, x_polar, theta_polar, z, f, p)
            else:
                x = y = z = f = p = None
                handle_command(command, x, y, z, f, p)

def handle_command(command, x_polar, theta_polar, z, f, p):
    if command == 'G0':
        print(f"Rapid move to X:{x_polar}, Theta:{theta_polar}°, Z:{z}, Maximum Velocity: {f}")
    elif command == 'G1':
        print(f"Linear move to X:{x_polar}, Theta:{theta_polar}°, Z:{z}, Maximum Velocity: {f}")
    elif command == 'G21':
        print("Set units to mm")
    elif command == 'G28':
        if x_polar is None and theta_polar is None and z is None:
            print("AutoHome all axes")
        else:
            axes_to_home = []
            if x_polar is not None:
                axes_to_home.append("X")
            if theta_polar is not None:
                axes_to_home.append("Theta")
            if z is not None:
                axes_to_home.append("Z")
            axes_str = ", ".join(axes_to_home)
            print(f"Auto home axis: {axes_str}")
    elif command == 'G60':
        print(f"Save position to slot S{number}")
    elif command == 'G61':
        print(f"Go to absolute coordinates stored in slot S{number}")
    elif command == 'G90':
        print("Use Absolute Positioning")
    elif command == 'G91':
        print("Use Relative Positioning")
    elif command == 'M0':
        print("Absolute STOP")
    elif command == 'M204':
        print(f"Setting Acceleration to: {p}")

# Example usage
file_path = '/home/evan/Downloads/3DBenchy-S3D-Kossel-v3.gcode'
process_gcode(file_path)

In [4]:
# import numpy as np
# from time import sleep
# from time import time
# import pyvisa
# import math

# status_flag = True
# flagisSet = 0
# # Initialize Resource Manager
# rm = pyvisa.ResourceManager()

# # Define the GPIB addresses for each axis
# AXIS_ADDRESSES = {
#     'X': 2,
#     'Theta': 3,
#     'Z': 4
# }


    

# def process_gcode(file_path):
#     flagisSet = 0
#     with open(file_path, 'r') as file:
#         for line in file:
#             line = line.strip()
#             if not line:
#                 continue

#             parts = line.split()
#             command = parts[0]
#             if parts[0] in ('G0', 'G1', 'G28', 'M204'):
#                 x = y = z = f = p = None
#                 for part in parts[1:]:
#                     if part.startswith('X'):
#                         x = float(part[1:])
#                     elif part.startswith('Y'):
#                         y = float(part[1:])
#                     elif part.startswith('Z'):
#                         z = float(part[1:])
#                     elif part.startswith('F'):
#                         f = float(part[1:])
#                     elif part.startswith('P'):
#                         p = float(part[1:])
                
#                 # Convert to polar coordinates if necessary
#                 x_polar, theta_polar = None, None
#                 if x is not None and y is not None:
#                     x_polar = math.sqrt(x**2 + y**2)  # Radial distance
#                     theta_polar = math.degrees(math.atan2(y, x))  # Angle in degrees
                
#                 handle_command(command, x_polar, theta_polar, z, f, p)
#             else:
#                 x = y = z = f = p = None
#                 handle_command(command, x, y, z, f, p)

# def open_axis(axis):
#     address = AXIS_ADDRESSES.get(axis)
#     if address is None:
#         raise ValueError(f"Invalid axis: {axis}")
#     return rm.open_resource(f'GPIB0::{address}::INSTR')

# def move_axis(axis, commands):
#     global flagisSet  # Declare flagisSet as global to modify it
    
#     if commands is not None:
#         for command in commands:
#             if command == 'B':
#                 axis_hp = open_axis(axis)
#                 axis_hp.timeout = 10000
#                 axis_hp.write_termination = '\r'
#                 axis_hp.read_termination = '\r'
#                 if flagisSet == 0: 
#                     print('hack')
#                     axis_hp.write(f'Z 1')
#                     axis_hp.write(f'S 40')
#                     axis_hp.write(f'F 10')
#                     axis_hp.write(f'A 0')
#                     axis_hp.write(f'R 100')
#                     flagisSet = 1
#                 else:
#                     axis_hp.write(f'{command}')
               
#             elif command == 'C':
#                 axis_hp.write(f'{command}')
#                 if axis_hp:
#                     axis_hp.close()
#             else:
#                 print('hi')
#                 axis_hp.write(f'{command}')

#     else:
#         print(f'no commands for {axis}')
        
# def handle_command(command, x_polar, theta_polar, z, f, p):
#     if command == 'G0':
#         print(f"Rapid move to X:{x_polar}, Theta:{theta_polar}°, Z:{z}, Maximum Velocity: {f}")

#         #NEED TO MAKE A CALCULATOR FOR CONVERTING THE VALUE OF THE x,theta,z and f given to the actual value given to the controller calc_coords()

#         x_commands = []
#         theta_commands = []
#         z_commands = []
#         # Heres where I make the bank of send_command commmands to control the different axes to move the stage head to the exact coordinates passed
#         if x_polar is not None:
#             x_commands.append(f'B')
#             if f is not None:
#                 x_commands.append(f'R {f}')
#             x_commands.append(f'A 0')
#             x_commands.append(f'N {x_polar}')
#             x_commands.append(f'C')
#         elif theta_polar is not None:
#             theta_commands.append(f'B')
#             if f is not None:
#                 theta_commands.append(f'R {f}')
#             theta_commands.append(f'P {theta_polar}')
#             theta_commands.append(f'C')
#         elif z is not None:
#             z_commands.append(f'B')
#             if f is not None:
#                 z_commands.append(f'R {f}')
#             z_commands.append(f'P {z}')
#             z_commands.append(f'C')
#         elif x_polar is None and theta_polar is None and z is None:
#             x_commands.append(f'R {f}')
#             theta_commands.append(f'R {f}')
#             z_commands.append(f'R {f}')

#     elif command == 'G1':
#         print(f"Linear move to X:{x_polar}, Theta:{theta_polar}°, Z:{z}, Maximum Velocity: {f}")
#         x_commands = []
#         theta_commands = []
#         z_commands = []
#         # Heres where I make the bank of send_command commmands to control the different axes to move the stage head to the exact coordinates passed
#         if x_polar is not None:
#             x_commands.append(f'B')
#             if f is not None:
#                 x_commands.append(f'R {f}')
#             x_commands.append(f'P {x_polar}')
#             x_commands.append(f'C')
#         elif theta_polar is not None:
#             theta_commands.append(f'B')
#             if f is not None:
#                 theta_commands.append(f'R {f}')
#             theta_commands.append(f'P {theta_polar}')
#             theta_commands.append(f'C')
#         elif z is not None:
#             z_commands.append(f'B')
#             if f is not None:
#                 z_commands.append(f'R {f}')
#             z_commands.append(f'P {z}')
#             z_commands.append(f'C')
#         elif x_polar is None and theta_polar is None and z is None:
#             x_commands.append(f'R {f}')
#             theta_commands.append(f'R {f}')
#             z_commands.append(f'R {f}')
#         move_axis('X', x_commands) #uncomment when using GPIB machine
#         move_axis('Theta', theta_commands)
#         move_axis('Z', z_commands)

#     elif command == 'G21':
#         print("Set units to mm")


#     elif command == 'G28':
#         if x_polar is None and theta_polar is None and z is None:
#             print("AutoHome all axes")
#         else:
#             axes_to_home = []
#             if x_polar is not None:
#                 axes_to_home.append("X")
#             if theta_polar is not None:
#                 axes_to_home.append("Theta")
#             if z is not None:
#                 axes_to_home.append("Z")
#             axes_str = ", ".join(axes_to_home)
#             print(f"Auto home axis: {axes_str}")


#     elif command == 'G60':
#         print(f"Save position to slot S{number}")
#     elif command == 'G61':
#         print(f"Go to absolute coordinates stored in slot S{number}")
#     elif command == 'G90':
#         print("Use Absolute Positioning")
#     elif command == 'G91':
#         print("Use Relative Positioning")
#     elif command == 'M0':
#         print("Absolute STOP")
#     elif command == 'M204':
#         print(f"Setting Acceleration to: {p}")
#         x_commands = []
#         theta_commands = []
#         z_commands = []
#         x_commands.append(f'B')
#         x_commands.append(f'S {p}')
#         x_commands.append(f'C')
#         theta_commands.append(f'B')
#         theta_commands.append(f'S {p}')
#         theta_commands.append(f'C')
#         z_commands.append(f'B')
#         z_commands.append(f'S {p}')
#         z_commands.append(f'C')
        

        



# # main()
# file_path = '/home/evan/GitPython/GCODE/Test.gcode'

# process_gcode(file_path)



Setting Acceleration to: 10.0
Linear move to X:200.0, Theta:0.0°, Z:0.0, Maximum Velocity: None
hack
hi


In [4]:
import numpy as np
from time import sleep
from time import time
from pyvisa import ResourceManager, constants, VisaIOError
from pyvisa.constants import EventType, StatusCode
import math

status_flag = True
flagisSet = 0
# Initialize Resource Manager
rm = pyvisa.ResourceManager()

# Define the GPIB addresses for each axis
AXIS_ADDRESSES = {
    'X': 2,
    'Theta': 4,
    'Z': 3
}

def axis_init(axis, axis_hp):
    if axis == 'X':
        axis_hp.write(f'Z 1')
        axis_hp.write(f'S 40')
        axis_hp.write(f'F 10')
        axis_hp.write(f'A 0')
        axis_hp.write(f'R 100')
    elif axis == 'Y':
        axis_hp.write(f'A 0')
        axis_hp.write(f'S 10')
        axis_hp.write(f'Z 1')
        axis_hp.write(f'F 10')
        axis_hp.write(f'R 18')
        
    elif axis == 'Z':
        axis_hp.write(f'R 60')
        axis_hp.write(f'Z 1')
        axis_hp.write(f'S 10')
        axis_hp.write(f'F 10')
        axis_hp.write(f'A 0')
def axis_home(axis, axis_hp):
    if axis == 'X':
        try:
            axis_hp.write(f'B') #when in limit it should try to get out and rehome
            axis_hp.write(f'R 50')
            axis_hp.write(f'Z 1')
            axis_hp.write(f'S 40')
            axis_hp.write(f'F 10')
            axis_hp.write(f'A 0')
            axis_hp.write(f'N 1')
            axis_hp.write(f'R 50')
            axis_hp.write(f'-')
            axis_hp.write(f'B')
            axis_hp.write(f'N 2600')
            axis_hp.write(f'G')
            axis_hp.write(f'C')
            axis_hp.write(f'R 100')
            stb = axis_hp.read_stb()
            while stb == 32:
                axis_hp.write(f'B')
                axis_hp.write(f'R 50')
                axis_hp.write(f'Z 1')
                axis_hp.write(f'S 40')
                axis_hp.write(f'F 10')
                axis_hp.write(f'A 0')
                axis_hp.write(f'N 1')
                axis_hp.write(f'R 50')
                axis_hp.write(f'-')
                axis_hp.write(f'B')
                axis_hp.write(f'N 2600')
                axis_hp.write(f'G')
                axis_hp.write(f'C')
                axis_hp.write(f'R 100')
                stb = axis_hp.read_stb()
            print(axis_hp.read_stb())
        except VisaIOError as e:
            if e.error_code == pyvisa.constants.StatusCode.error_system_error:
                print("System error encountered during read_stb().")
            else:
                print(f"Unexpected VISA error: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
    if axis == 'Y':
        axis_hp.write(f'-')
        axis_hp.write(f'B')
        axis_hp.write(f'N 400')
        axis_hp.write(f'G')
        axis_hp.write(f'C')
        axis_hp.write(f'R 60')
        axis_hp.write(f'Z 1')
        axis_hp.write(f'S 10')
        axis_hp.write(f'F 10')
        axis_hp.write(f'A 0')
        axis_hp.write(f'N 1')
        axis_hp.write(f'-')
        axis_hp.write(f'B')
        axis_hp.write(f'N 7000')
        axis_hp.write(f'G')
        axis_hp.write(f'C')
        print('Y axis homing constants')
    if axis == 'Z':
        axis_hp.write(f'A 0')
        axis_hp.write(f'S 10')
        axis_hp.write(f'Z 1')
        axis_hp.write(f'F 10')
        axis_hp.write(f'R 18')
        axis_hp.write(f'N 1')
        axis_hp.write(f'R 18')
        axis_hp.write(f'B')
        axis_hp.write(f'+')
        axis_hp.write(f'N 720')
        axis_hp.write(f'G')
        axis_hp.write(f'+')
        axis_hp.write(f'N 720')
        axis_hp.write(f'G')
        axis_hp.write(f'Q')
        axis_hp.write(f'E')
        axis_hp.write(f'Q')
        axis_hp.write(f'-')
        axis_hp.write(f'N 3320')
        axis_hp.write(f'G')
        axis_hp.write(f'C')
        axis_hp.write(f'R 100')
        axis_hp.write(f'+')
        axis_hp.write(f'B')
        axis_hp.write(f'N 1800')
        axis_hp.write(f'G')
        axis_hp.write(f'C')
        
def srq_handler(session, event_type, context, user_handle):
    """
    SRQ event handler: This function is called when the SRQ event is triggered.
    """
    stb = session.read_stb()[0]  # Read the status byte
    if stb & 0x01:  # Check if the first bit of the status byte is set
        print(f"SRQ received: Status Byte = {stb}")
        return True
    return False

def wait_for_srq(axis_hp):
    """
    Wait for the SRQ event by polling the status byte.
    """
    try:
        print("Waiting for SRQ...")

        while True:
            stb = axis_hp.read_stb()  # Read the status byte
            if stb & 0x40:  # Check if the SRQ bit (6th bit) of the status byte is set
                print(f"SRQ received: Status Byte = {stb}")
                break
            sleep(0.1)  # Small delay to avoid busy-waiting

    except VisaIOError as e:
        print(f"VISA error: {e}")
    finally:
        pass
        
def process_gcode(file_path):
    flagisSet = 0
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue

            parts = line.split()
            command = parts[0]
            if parts[0] in ('G0', 'G1', 'G28', 'M204'):
                x = y = z = f = p = None
                for part in parts[1:]:
                    if part.startswith('X'):
                        x = float(part[1:])
                    elif part.startswith('Y'):
                        y = float(part[1:])
                    elif part.startswith('Z'):
                        z = float(part[1:])
                    elif part.startswith('F'):
                        f = float(part[1:])
                    elif part.startswith('P'):
                        p = float(part[1:])
                
                # Convert to polar coordinates if necessary
                x_polar, theta_polar = None, None
                if x is not None and y is not None:
                    x_polar = math.sqrt(x**2 + y**2)  # Radial distance
                    theta_polar = math.degrees(math.atan2(y, x))  # Angle in degrees
                
                handle_command(command, x_polar, theta_polar, z, f, p)
            else:
                x = y = z = f = p = None
                handle_command(command, x, y, z, f, p)

def open_axis(axis):
    address = AXIS_ADDRESSES.get(axis)
    if address is None:
        raise ValueError(f"Invalid axis: {axis}")
        
    return rm.open_resource(f'GPIB0::{address}::INSTR')

def move_axis(axis, commands):
    global flagisSet  # Declare flagisSet as global to modify it
    
    if commands is not None:
        for command in commands:
            if command == 'B':
                axis_hp = open_axis(axis)
                axis_hp.timeout = 10000
                axis_hp.write_termination = '\r'
                axis_hp.read_termination = '\r'
                
                
#                 # Type of event we want to be notified about
#                 event_type = constants.EventType.service_request
#                 # Mechanism by which we want to be notified
#                 event_mech = constants.EventMechanism.queue

#                 axis_hp.enable_event(event_type, event_mech)


                if flagisSet == 0: 
                    axis_home(axis, axis_hp)
                    axis_init(axis, axis_hp)
                    print(f'initialize axis: {axis}')
                    axis_hp.write(f'B')
                    print('B command')
                    flagisSet = 1
                else:
                    axis_hp.write(f'{command}')
                    print('B command')
               
            elif command == 'C':
                axis_hp.write(f'{command}')
#                 print(axis_hp.read_stb())
                print('C command')
                if axis_hp:
                    axis_hp.close()
            else:
                print(f'{command}')
                axis_hp.write(f'{command}')
#                 if status_flag:
#                     status = axis_hp.stb
#                     print(f"Status for axis {axis} after '{command}': {parse_status(status)}")
    else:
        print(f'no commands for {axis}')
        
def handle_command(command, x_polar, theta_polar, z, f, p):
    if command == 'G0':
        print(f"Rapid move to X:{x_polar}, Theta:{theta_polar}°, Z:{z}, Maximum Velocity: {f}")

        #NEED TO MAKE A CALCULATOR FOR CONVERTING THE VALUE OF THE x,theta,z and f given to the actual value given to the controller calc_coords()

        x_commands = []
        theta_commands = []
        z_commands = []
        # Heres where I make the bank of send_command commmands to control the different axes to move the stage head to the exact coordinates passed
        if x_polar is not None:
            x_commands.append(f'B')
            if f is not None:
                x_commands.append(f'R {f}')
            x_commands.append(f'P {x_polar}')
            x_commands.append(f'C')
        elif theta_polar is not None:
            theta_commands.append(f'B')
            if f is not None:
                theta_commands.append(f'R {f}')
            theta_commands.append(f'P {theta_polar}')
            theta_commands.append(f'C')
        elif z is not None:
            z_commands.append(f'B')
            if f is not None:
                z_commands.append(f'R {f}')
            z_commands.append(f'P {z}')
            z_commands.append(f'C')
        elif x_polar is None and theta_polar is None and z is None:
            x_commands.append(f'R {f}')
            theta_commands.append(f'R {f}')
            z_commands.append(f'R {f}')
        # move_axis('X', x_commands) #uncomment when using on GPIB machine
        # move_axis('Theta', theta_commands)
        # move_axis('Z', z_commands)

    elif command == 'G1':
        print(f"Linear move to X:{x_polar}, Theta:{theta_polar}°, Z:{z}, Maximum Velocity: {f}")
        x_commands = []
        theta_commands = []
        z_commands = []
        # Heres where I make the bank of send_command commmands to control the different axes to move the stage head to the exact coordinates passed
        if x_polar is not None:
            x_commands.append(f'B')
            if f is not None:
                x_commands.append(f'R {f}')
            x_commands.append(f'P {x_polar}')
            x_commands.append(f'C')
        elif theta_polar is not None:
            theta_commands.append(f'B')
            if f is not None:
                theta_commands.append(f'R {f}')
            theta_commands.append(f'P {theta_polar}')
            theta_commands.append(f'C')
        elif z is not None:
            z_commands.append(f'B')
            if f is not None:
                z_commands.append(f'R {f}')
            z_commands.append(f'P {z}')
            z_commands.append(f'C')
        elif x_polar is None and theta_polar is None and z is None:
            x_commands.append(f'R {f}')
            theta_commands.append(f'R {f}')
            z_commands.append(f'R {f}')
        move_axis('X', x_commands) #uncomment when using GPIB machine
        move_axis('Theta', theta_commands)
        move_axis('Z', z_commands)

    elif command == 'G21':
        print("Set units to mm")


    elif command == 'G28':
        if x_polar is None and theta_polar is None and z is None:
            print("AutoHome all axes")
        else:
            axes_to_home = []
            if x_polar is not None:
                axes_to_home.append("X")
            if theta_polar is not None:
                axes_to_home.append("Theta")
            if z is not None:
                axes_to_home.append("Z")
            axes_str = ", ".join(axes_to_home)
            print(f"Auto home axis: {axes_str}")


    elif command == 'G60':
        print(f"Save position to slot S{number}")
    elif command == 'G61':
        print(f"Go to absolute coordinates stored in slot S{number}")
    elif command == 'G90':
        print("Use Absolute Positioning")
    elif command == 'G91':
        print("Use Relative Positioning")
    elif command == 'M0':
        print("Absolute STOP")
    elif command == 'M204':
        print(f"Setting Acceleration to: {p}")
        x_commands = []
        theta_commands = []
        z_commands = []
        x_commands.append(f'B')
        x_commands.append(f'S {p}')
        x_commands.append(f'C')
        theta_commands.append(f'B')
        theta_commands.append(f'S {p}')
        theta_commands.append(f'C')
        z_commands.append(f'B')
        z_commands.append(f'S {p}')
        z_commands.append(f'C')
        move_axis('X', x_commands) #uncomment when using GPIB machine
        move_axis('Theta', theta_commands)
        move_axis('Z', z_commands)
        

        



# main()
file_path = '/home/evan/GitPython/GCODE/Test.gcode'
# rm = pyvisa.ResourceManager() #this is a test on the use of stb and SRQ event handling
# axis_hp = rm.open_resource('GPIB0::2::INSTR')

# wait_for_srq(axis_hp)

# process_gcode(file_path) #X100 = 2.5 cm
axis = 'Z'
axis_hp = open_axis(axis)
axis_hp.timeout = 10000
axis_hp.write_termination = '\r'
axis_hp.read_termination = '\r'
axis_hp.write(f'B')
axis_hp.write(f'R 180')
axis_hp.write(f'Z 1')
axis_hp.write(f'S 10')
axis_hp.write(f'F 10')
axis_hp.write(f'A 0')
# axis_hp.write(f'P 200')
axis_hp.write(f'C')
for i in range(10): #change Z init rates back to normal
    axis_hp.write(f'-')
    axis_hp.write(f'B')
    axis_hp.write(f'N 400')
    axis_hp.write(f'G')
    axis_hp.write(f'C')
    axis_hp.write(f'R 20')
    axis_hp.write(f'Z 2')
    axis_hp.write(f'S 10')
    axis_hp.write(f'F 30')
    axis_hp.write(f'A 0')
    axis_hp.write(f'N 1')
    axis_hp.write(f'-')
    axis_hp.write(f'B')
    axis_hp.write(f'N 7000')
    axis_hp.write(f'G')
    axis_hp.write(f'C')

print(f'initialize axis: {axis}')



initialize axis: Z
